Merge data dari Google Drive dlu mank wkwk

In [ ]:
# Merge Data dari Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Libraries Import ae smua wkwk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
import keras
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

Extrack file zip yang dari Google Drive. Atur dlu direktorinya yakk wkwk

In [ ]:
# Extract Zip File yang ada di drive ke folder di Colab 
import zipfile
local_zip = '/content/drive/My Drive/Copy of Copy of shopee-product-detection-dataset.zip' # Spesifikasiin direktori file zip nya
zip_ref = zipfile.ZipFile(local_zip, mode = 'r', allowZip64 = True)
zip_ref.extractall('/tmp') # nanti file extractannya bakal ada di folder ini
zip_ref.close()

In [ ]:
# Overview data train
data = pd.read_csv('/tmp/train.csv')
data

In [ ]:
# Ploting
plt.figure(figsize=(10,10))
sns.countplot(x = 'category', data = data)
plt.title('Figure Size')
plt.show()

Base Dir ini folder yang isinya data train yakk

In [ ]:
base_dir = '/tmp/train/train' # base dir buat data trainnya

Load Tranfer Learning. Disini gw make MobileNet. coba pek yang lain dah
https://keras.io/api/applications/

In [ ]:
# Load Tranfer Learning MobileNet
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(42,activation='softmax')(x) #final layer with softmax activation

In [ ]:
model_1=Model(inputs=base_model.input,outputs=preds)

Ini gw gak tau sumpah buat apa trainable"an awkwkwk

In [ ]:
for layer in model_1.layers[:20]:
    layer.trainable=False
for layer in model_1.layers[20:]:
    layer.trainable=True

Image data generator buat load data Imagenya dari base_dir. Pke vaidation Split buat ngebagi datanya buat ngukur akurasinya nanti.
Itu gw color modenya pke yang RGB. Coba pke yang lain deh kyk grayscale gtu" kyknya kaurasinya bakal bagus deh :v

In [ ]:
train_datagen=ImageDataGenerator(validation_split=0.25,
                                 preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory(base_dir,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 subset='training',
                                                 shuffle=True)
valid_generator=train_datagen.flow_from_directory(base_dir,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 subset='validation',
                                                 shuffle=True)

Set Compile model. dsini gw optimizernya pke Adam. Coba pke kyk SGD ato Adamax ato yang lain deh wkwk

In [ ]:
model_1.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

Ngefit modelnya deh keknya klo ini kgk usah di ubah. epochnya bisa kalian ubah. gw dstu pke 1 doang karena 1 aja udah 3 setengah jaman apalagi klo 10 wkwk.
tapi nanti kita perlu run lagi ini karena seenggaknya epochnya itu 20an biar dapet akurasi bagus. oh iyak klo bsa jangan lebih dari 3 yak karena google colab cuman nyedian waktu sampe 12 jam nanti klo lewat dari 12 jam lu lagi ngerun belom ngesave modelnya dijamin 100% kesel wkwk

In [ ]:
step_size_train=train_generator.n//train_generator.batch_size
model_1.fit(train_generator, 
            validation_data= valid_generator,
            steps_per_epoch=step_size_train,
            epochs=1)

Save modelnya jangan lupa baru run lagi fitnya wkwk

In [ ]:
model_1.save('drive/My Drive/model_saved_TL.h5')

klo mau load model yang udah ada pke syntax di bawah :v

In [ ]:
from keras.models import load_model
model_1 = load_model('drive/My Drive/model_saved_TL.h5')

Dimas Bau Bawank